# 入門編！Pythonで位置データの分析を体験しよう!

## 概要

- 1. データを取得する
    - 置いてあるcsvファイルを取得するコードを作成する
    - それで10個のファイルを取り、1つのファイルに落とし込む
- 2. データを加工する
    - 位置データとして扱えるようにデータ型を変える
    - テーブルデータをpandas -> geopandas に
- 3. データを可視化する
    - 全体の行動や個人の行動を確認する

2と3を何度か繰り返します。

コードはSHIFT + Enter もしくはコード左にある再生ボタンを押すと、動きます。

## 利用するデータ

- International Semantic Web Conference 2016 というイベントで取得された10人分の行動データ(csvファイル形式)
- [GPS trajectory linked data project](https://github.com/koujikozaki/GPS2LOD)
- [大阪電気通信大学　古崎教授が集められたデータ](https://www.osakac.ac.jp/labs/kozaki/)
- ライセンス [CC 4.0](https://creativecommons.org/licenses/by/4.0/deed.ja)

## 質問はSlidoに投稿ください！
[https://app.sli.do/event/gyunfwnp](https://app.sli.do/event/gyunfwnp)    
![](https://i.gyazo.com/1ddaff033507e66e1b85677dbe6749e7.png)


## まず必要なライブラリ/パッケージをインストール/インポートします
- ライブラリ/パッケージは前もって作られている、何かやりたいことを簡単に処理してくれるプログラムが前もってある
- このようなプログラムが多いことがPythonの特徴


In [25]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal &> /dev/null
# Install rtree - Geopandas requirment
!apt install python3-rtree &> /dev/null
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git &> /dev/null
# Install descartes - Geopandas requirment
!pip install descartes &> /dev/null
!pip install plotly -U &> /dev/null

�R�}���h�̍\��������Ă��܂��B
�R�}���h�̍\��������Ă��܂��B
�R�}���h�̍\��������Ă��܂��B
�R�}���h�̍\��������Ă��܂��B
�R�}���h�̍\��������Ă��܂��B


In [26]:
import time # 時間をコントロール
from typing import List 
from datetime import datetime, date # 日時をコントロール
import requests # requests: https://docs.python-requests.org/en/latest/
import pandas as pd # pandas: 表データを扱う: https://pandas.pydata.org/
import geopandas as gpd # geopandas: 位置データを含む表データを扱う: https://geopandas.org/
import folium # folium: 位置データを可視化する: http://python-visualization.github.io/folium/
from folium import plugins
import shapely.geometry # shapely: 位置データを扱う: https://shapely.readthedocs.io/en/stable/

ModuleNotFoundError: No module named 'geopandas'

## まずはデータを取得するための準備を行います

- 次のディレクトリに今回利用するファイルがあります
- https://github.com/koujikozaki/GPS2LOD/tree/GPS2LODforDP/CSV
- https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user1.csv
- ↑のURLをクリックすると、ファイルが見れます
- このようなファイルが10個あります。
- まずは、このURLを10個作成し、リストに格納します

In [3]:
# URLを作り、リストに格納する

url_list = list()

for i in range(1, 11):
    csv_url = f'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user{i}.csv'
    url_list.append(csv_url)

In [4]:
# url_list に10個のURLが格納されている

url_list

['https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user1.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user2.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user3.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user4.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user5.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user6.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user7.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user8.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user9.csv',
 'https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user10.csv']

In [5]:
# 繰り返し作業を行うのにループを使う

for i in range(1, 11):
    print(i)

1
2
3
4
5
6
7
8
9
10


### 番号で指定すると、リストから欲しいURLが取得できます
- 注意点: 一番初めのものを0から始める

In [6]:
print(url_list[0])
print(url_list[3])

https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user1.csv
https://raw.githubusercontent.com/koujikozaki/GPS2LOD/GPS2LODforDP/CSV/user4.csv


# データを取得する(1人分)

- CSVファイルを読み込みます
- pandasのread_csv関数で読み込めます
- pandas: Pythonを使ってエクセルのようなデータを処理するときによく使われるライブラリ

In [7]:
df = pd.read_csv(url_list[0])

In [8]:
# データの概要を見る
df.head()

,Date,Time,Latitude,Longitude,Altitude,Speed,Course,Type,Distance,Essential
0,2016/10/17,11:18:05,34.666027,135.213867,71.92,468.0,0,-2,0.00,1
1,2016/10/17,11:24:25,34.665825,135.214401,-27.23,972.0,114,0,53.76,1
2,2016/10/17,12:04:03,34.665825,135.214401,-27.23,972.0,0,-4,0.00,1
3,2016/10/17,12:05:39,34.665886,135.214172,177.23,468.0,0,-2,0.00,1
4,2016/10/17,12:06:46,34.665756,135.214111,151.22,576.0,201,0,15.49,1


In [9]:
# データの概要を見る
# カラムの先頭にスペースが含まれる -> あとから直す

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        707 non-null    object 
 1    Time       707 non-null    object 
 2    Latitude   707 non-null    float64
 3    Longitude  707 non-null    float64
 4    Altitude   707 non-null    float64
 5    Speed      707 non-null    float64
 6    Course     707 non-null    int64  
 7    Type       707 non-null    int64  
 8    Distance   707 non-null    float64
 9    Essential  707 non-null    int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 55.4+ KB


### データの中身を確認
- 日付、時間、緯度、経度、高さ、スピードなどがあるデータ


In [10]:
# 違うデータを読み込んでみる
df1 = pd.read_csv(url_list[1])

In [11]:
df1.head()

,Date,Time,Latitude,Longitude,Altitude,Speed,Course,Type,Distance,Essential
0,2016/10/17,11:00:39,34.666958,135.220444,532.75,15624.0,0,-2,0.00,1
1,2016/10/17,11:03:26,34.666584,135.213882,108.38,1044.0,266,0,601.57,1
2,2016/10/17,11:13:02,34.666126,135.213898,78.77,1152.0,178,0,50.95,1
3,2016/10/17,11:14:08,34.666164,135.213928,106.20,5508.0,32,0,5.04,1
4,2016/10/17,11:15:15,34.665009,135.208344,-192.81,0.0,255,-4,526.59,1


### データの中身
- データごとに時間はまちまちであることが分かります
- 実際は取得データを加工する前に、色々調べるのですが、その辺りは時間がかかるので、省略します

## データをいったん可視化する
- まずは可視化
- どこのデータか見てみる

In [12]:
center_lat = df1[' Latitude'].median() # 中央値を取る
center_lon = df1[' Longitude'].median() # 中央値を取る

m = folium.Map(location=[center_lat, center_lon]) # 地図を作る
folium.Marker(location=[center_lat, center_lon]).add_to(m) # 地図に中央値の場所をマーカーでのせる
m # 地図の表示

NameError: name 'folium' is not defined

### データ確認
- イベントサイトによると、神戸国際会議場で行われるとある。
- 中央値が会場にプロットされているので、正解っぽい
- https://iswc2016.semanticweb.org/pages/attending/venue.html

## すべてのデータを合わせたデータを作る
- 先ほど見たのは一人分でした
- 10人分あるので、10人分のデータを使えるようにします
- もとのデータだと、人の区別がつかないので、人の番号をふります
    - number という属性を作ります
- 時間を一致するように作りたいので、5分おきのデータにします
    - Pythonで扱える日時にする
    - それを使ってresamplingする
- データは簡単に使えない（かなりきれいなデータでも加工が必要となります）

## データを使いやすいように整える
- 日付と時間はPythonから扱いやすいよう datetime 型
- データの頻度がバラバラなので、30分おきのデータとする
- 今回使う経度緯度、速度のデータ以外は落とし、データのない部分も落とす

In [13]:
df.head()

,Date,Time,Latitude,Longitude,Altitude,Speed,Course,Type,Distance,Essential
0,2016/10/17,11:18:05,34.666027,135.213867,71.92,468.0,0,-2,0.00,1
1,2016/10/17,11:24:25,34.665825,135.214401,-27.23,972.0,114,0,53.76,1
2,2016/10/17,12:04:03,34.665825,135.214401,-27.23,972.0,0,-4,0.00,1
3,2016/10/17,12:05:39,34.665886,135.214172,177.23,468.0,0,-2,0.00,1
4,2016/10/17,12:06:46,34.665756,135.214111,151.22,576.0,201,0,15.49,1


In [14]:
df['dt'] = df['Date'] + ' ' + df[' Time']
df['dt'] = pd.to_datetime(df['dt']) 
df['number'] = 'number1'

In [ ]:
df.head()

,Date,Time,Latitude,Longitude,Altitude,Speed,Course,Type,Distance,Essential,dt,number
0,2016/10/17,11:18:05,34.666027,135.213867,71.92,468.0,0,-2,0.00,1,2016-10-17 11:18:05,number1
1,2016/10/17,11:24:25,34.665825,135.214401,-27.23,972.0,114,0,53.76,1,2016-10-17 11:24:25,number1
2,2016/10/17,12:04:03,34.665825,135.214401,-27.23,972.0,0,-4,0.00,1,2016-10-17 12:04:03,number1
3,2016/10/17,12:05:39,34.665886,135.214172,177.23,468.0,0,-2,0.00,1,2016-10-17 12:05:39,number1
4,2016/10/17,12:06:46,34.665756,135.214111,151.22,576.0,201,0,15.49,1,2016-10-17 12:06:46,number1


In [15]:
df = df.set_index('dt')

In [16]:
# データを30分おきにする。最後の値を取得する
df1 = df.resample('30T').last()

In [17]:
df1.head(20)

,Date,Time,Latitude,Longitude,Altitude,Speed,Course,Type,Distance,Essential,number
dt,,,,,,,,,,,
2016-10-17 11:00:00,2016/10/17,11:24:25,34.665825,135.214401,-27.23,972.0,114.0,0.0,53.76,1.0,number1
2016-10-17 11:30:00,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2016-10-17 12:00:00,2016/10/17,12:29:40,34.665089,135.214233,-111.74,396.0,141.0,0.0,87.92,1.0,number1
2016-10-17 12:30:00,2016/10/17,12:44:23,34.668037,135.212723,130.69,9108.0,356.0,0.0,51.78,1.0,number1
2016-10-17 13:00:00,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2016-10-17 13:30:00,2016/10/17,13:46:02,34.665871,135.212769,11.17,1152.0,225.0,0.0,82.12,1.0,number1
2016-10-17 14:00:00,2016/10/17,14:01:04,34.665062,135.211563,378.18,0.0,230.0,0.0,142.33,1.0,number1
2016-10-17 14:30:00,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2016-10-17 15:00:00,2016/10/17,15:23:18,34.666470,135.212982,164.42,6552.0,137.0,0.0,10.27,1.0,number1


In [18]:
# データがないところはいらないということで、落とす
df1 = df.resample('30T').last().dropna(how='all')

In [19]:
# 今回使うデータだけをピックアップする
# 今回は経度緯度、速度を利用します。
df1 = df1[[' Latitude', ' Longitude', ' Speed']].reset_index()

In [20]:
df1.tail()

,dt,Latitude,Longitude,Speed
87,2016-10-22 00:00:00,34.683933,135.190552,864.0
88,2016-10-22 10:30:00,34.689369,135.196274,40320.0
89,2016-10-22 11:00:00,34.695107,135.195343,1188.0
90,2016-10-22 11:30:00,34.695335,135.193512,0.0
91,2016-10-22 12:30:00,34.698357,135.193542,0.0


## データを作成する部分を関数にまとめる
- URLを指定すると、そのデータを前処理してくれるものを作る
- ↑の作業のまとめ
- それを作った後、10人分のデータをまとめる関数も作成する

In [21]:
def make_personal_dataframe(url: str, num: int):
    '''
    URLを渡すとデータを作成してくれる関数
    Params:
        url: csvファイルのあるURL
        num: データの人を区別するための数
    Returns:
        df: 移動データ（30分刻みのデータ）
    '''
    df = pd.read_csv(url)
    df['dt'] = df['Date'] + ' ' + df[' Time']
    df['dt'] = pd.to_datetime(df['dt'])
    df['number'] = f'number{num}'
    df = df.set_index('dt')
    df = df.resample('30T').last().dropna(how='all')
    df = df[[' Latitude', ' Longitude', ' Speed', 'number']]
    cols = [col.replace(' ', '') for col in df.columns]
    df.columns = cols
    df = df.reset_index()
    new_columns = [col.replace(' ', '') for col in df.columns]
    df.columns = new_columns
    return df

def main(url_list: List[str]):
    '''
    すべてのcsvファイルを取得する関数
    Params:
        url_list: csvファイルのURLを格納したリスト
    Returns:
        data: すべてのファイルのデータを返す
    '''
    data = pd.DataFrame()
    for num, url in enumerate(url_list):
        df = make_personal_dataframe(url, num+1)
        data = pd.concat([data, df])
        print(f'{num}件目終了しました')
        time.sleep(3) # 3秒休む: データを取得してサーバに過剰な負担をかけるのを避ける
    data = data.reset_index(drop=True)
    return data

In [23]:
df = make_personal_dataframe(url_list[0], 1)
print(df.columns)
df.head()

Index(['dt', 'Latitude', 'Longitude', 'Speed', 'number'], dtype='object')


,dt,Latitude,Longitude,Speed,number
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1


In [24]:
df.to_csv('./data/behavior_data.csv', index=None)

### 10人分のデータを作成する
- 先ほど作成した関数を実行して、10人分のデータを作成する
- main関数にURLを格納したリストを渡す

In [ ]:
df = main(url_list)

0件目終了しました
1件目終了しました
2件目終了しました
3件目終了しました
4件目終了しました
5件目終了しました
6件目終了しました
7件目終了しました
8件目終了しました
9件目終了しました


In [ ]:
df

,dt,Latitude,Longitude,Speed,number
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1
...,...,...,...,...,...
758,2016-10-22 00:00:00,34.684116,135.188736,936.0,number10
759,2016-10-22 10:30:00,34.685192,135.197800,14400.0,number10
760,2016-10-22 11:00:00,34.694668,135.195465,2160.0,number10
761,2016-10-22 11:30:00,34.696968,135.194412,0.0,number10


## 経度緯度を位置データとして扱う
- 実は今の状態では位置データとして扱っていません
- 緯度と経度がバラバラです
- pythonでは位置データを[Shapely](https://shapely.readthedocs.io/en/stable/)というライブラリで扱います。
    - ちなみに、位置データはPoint, Line, Polygonなどで扱います。それぞれは次のような表現となります
    - Pointは1つの点
    - Lineは線
    - Polygonは表面
- 移動点をPointとします
- 位置データを扱うので、DataFrame -> GeoDataFrame に変換します
- GeoDataFrame は geopandas のデータ型で、これで位置データが扱いやすくなります
- そしてCRS(座標系)を設定します
    - 今回は世界測地系 WGS84 / 日本測地系2011を利用
    - [ESRI ジャパン: 座標系とは](https://www.esrij.com/gis-guide/coordinate-and-spatial/coordinate-system/)
    - [ESRI ジャパン: 日本で使用される座標系](https://www.esrij.com/gis-guide/coordinate-and-spatial/coordinate-system-japan/)

In [ ]:
df['geometry'] = df.apply(lambda x: shapely.geometry.Point(x[2], x[1]), axis=1)

In [ ]:
df

,dt,Latitude,Longitude,Speed,number,geometry
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1,POINT (135.214401 34.665825)
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1,POINT (135.214233 34.665089)
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1,POINT (135.212723 34.668037)
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1,POINT (135.212769 34.665871)
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1,POINT (135.211563 34.665062)
...,...,...,...,...,...,...
758,2016-10-22 00:00:00,34.684116,135.188736,936.0,number10,POINT (135.188736 34.684116)
759,2016-10-22 10:30:00,34.685192,135.197800,14400.0,number10,POINT (135.1978 34.685192)
760,2016-10-22 11:00:00,34.694668,135.195465,2160.0,number10,POINT (135.195465 34.694668)
761,2016-10-22 11:30:00,34.696968,135.194412,0.0,number10,POINT (135.194412 34.696968)


In [ ]:
# 位置データを扱いやすくするため、GeoDataFrameに変換します
gdf = gpd.GeoDataFrame(df)
# 座標系を与える WGS84
gdf = gdf.set_crs('EPSG:4326')

In [ ]:
gdf

,dt,Latitude,Longitude,Speed,number,geometry
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1,POINT (135.21440 34.66582)
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1,POINT (135.21423 34.66509)
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1,POINT (135.21272 34.66804)
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1,POINT (135.21277 34.66587)
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1,POINT (135.21156 34.66506)
...,...,...,...,...,...,...
758,2016-10-22 00:00:00,34.684116,135.188736,936.0,number10,POINT (135.18874 34.68412)
759,2016-10-22 10:30:00,34.685192,135.197800,14400.0,number10,POINT (135.19780 34.68519)
760,2016-10-22 11:00:00,34.694668,135.195465,2160.0,number10,POINT (135.19546 34.69467)
761,2016-10-22 11:30:00,34.696968,135.194412,0.0,number10,POINT (135.19441 34.69697)


In [ ]:
# Geodataframe にCRSが設定される
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## 全員の行動を可視化する
- 課題: 一人ずつの色を変えたい

In [ ]:
ps_color = {'number1': 'blue',
            'number2': 'green',
            'number3': 'white',
            'number4': 'gray',
            'number5': 'black',
            'number6': 'red',
            'number7': 'orange',
            'number8': 'pink',
            'number9': 'purple',
            'number10': 'lime'
            }
ps_color['number1']

'blue'

In [ ]:
center = shapely.geometry.MultiPoint(df['geometry'].values).centroid

map = folium.Map(location=[center.y, center.x])

for i in df.index:
    num = df.loc[i, 'number']
    selected_color=ps_color[num]
    posi = df.loc[i, 'geometry']
    folium.Marker([posi.y, posi.x], popup=df.loc[i, 'number'], icon=folium.Icon(color=selected_color)).add_to(map)

map

## 1人だけの行動を観察したい場合
- 要素を抽出する

In [ ]:
selected_num = 4
selected_data = f'number{selected_num}'

selected_df = df[df['number'] == selected_data]
center = shapely.geometry.MultiPoint(selected_df['geometry'].values).centroid

map = folium.Map(location=[center.y, center.x])
for i in selected_df.index:

    posi = selected_df.loc[i, 'geometry']
    folium.Marker([posi.y, posi.x], popup=str(df.loc[i, 'dt'])).add_to(map)

map

## ある日の行動を観察してみる
- 線(LineString)にしてかんさつする


In [ ]:
selected_num = 3
selected_data = f'number{selected_num}'
selected_df = df[df['number'] == selected_data]

date = '20161022'

selected_df1 = selected_df.set_index('dt')
selected_df1 = selected_df1[date]
selected_line = shapely.geometry.LineString(selected_df1['geometry'].values)
center = selected_line.centroid
map = folium.Map(location=[center.y, center.x])
folium.GeoJson(selected_line).add_to(map)
map

## ヒートマップで行動を観察する

In [ ]:
matrix_data = df[['Latitude', 'Longitude']].values

m = folium.Map([df.iloc[0, -1].y, df.iloc[0, -1].x])
m.add_child(plugins.HeatMap(matrix_data))
m

In [ ]:
# 全体の選択した日のデータを作成

def date_select(df, date):
    dff = df.set_index('dt')
    data = pd.DataFrame()
    for num in dff['number'].unique():
        sel_df = dff[dff['number'] == num]
        sel_df = sel_df[date]
        data = pd.concat([data, sel_df])
    return data

# 日付を指定してヒートマップを見る

In [ ]:
# 日を選ぶ
sel_date = '20161022'

date_df = date_select(df, sel_date)
center = shapely.geometry.MultiPoint(date_df['geometry'].values).centroid
matrix_data = date_df[['Latitude', 'Longitude']].values

m = folium.Map([center.y, center.x])
m.add_child(plugins.HeatMap(matrix_data))
m

## 一時点を指定して、スピードを比較する
- ある瞬間の各自のスピードを比較する
- 在庫の管理などもできる
- 動いているもののリアルタイム管理とかもできる

In [ ]:
dtime = datetime(2016,10,18,13,0)

d_df = df[df['dt'] == dtime]

center = shapely.geometry.MultiPoint(d_df['geometry'].values).centroid

map = folium.Map([center.y, center.x], zoom_start=14)

for i in d_df.index:
    posi = d_df.loc[i, 'geometry']
    speed = d_df.loc[i, 'Speed']/100
    popup = d_df.loc[i, 'number']
    folium.Circle([posi.y, posi.x], radius=speed, popup=popup).add_to(map)

map

### CRSを変えて、各個人の移動距離を計測する

- CRSをWGS84 -> JGD2011 に変換して距離を出す
- 一人のある日の移動距離をとる

In [ ]:
# crsを変更する
# ある位置からの距離を示す
df_changed = gdf.to_crs('EPSG:6673')

In [ ]:
df_changed

,dt,Latitude,Longitude,Speed,number,geometry
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1,POINT (80749.735 -147654.354)
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1,POINT (80735.052 -147736.132)
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1,POINT (80593.800 -147410.325)
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1,POINT (80600.114 -147650.558)
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1,POINT (80490.364 -147741.263)
...,...,...,...,...,...,...
758,2016-10-22 00:00:00,34.684116,135.188736,936.0,number10,POINT (78380.243 -145645.647)
759,2016-10-22 10:30:00,34.685192,135.197800,14400.0,number10,POINT (79209.772 -145519.194)
760,2016-10-22 11:00:00,34.694668,135.195465,2160.0,number10,POINT (78986.811 -144469.873)
761,2016-10-22 11:30:00,34.696968,135.194412,0.0,number10,POINT (78888.152 -144215.563)


In [ ]:
df_changed.crs

<Projected CRS: EPSG:6673>
Name: JGD2011 / Japan Plane Rectangular CS V
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Japan - onshore - Honshu between approximately 133°15'E and 135°10'E - Hyogo-ken; Tottori-ken; Okayama-ken.
- bounds: (133.13, 34.13, 135.47, 35.71)
Coordinate Operation:
- name: Japan Plane Rectangular CS zone V
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
df_changed['dt_date'] = df_changed['dt'].map(lambda x: x.date())
date_list = list(df_changed['dt_date'].unique())
date_list

[datetime.date(2016, 10, 17),
 datetime.date(2016, 10, 18),
 datetime.date(2016, 10, 19),
 datetime.date(2016, 10, 20),
 datetime.date(2016, 10, 21),
 datetime.date(2016, 10, 22),
 datetime.date(2016, 10, 16)]

In [ ]:
# 日付のデータを取得する
df_changed[df_changed['dt_date'] == date(2016,10,17)]

,dt,Latitude,Longitude,Speed,number,geometry,dt_date
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1,POINT (80749.735 -147654.354),2016-10-17
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1,POINT (80735.052 -147736.132),2016-10-17
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1,POINT (80593.800 -147410.325),2016-10-17
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1,POINT (80600.114 -147650.558),2016-10-17
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1,POINT (80490.364 -147741.263),2016-10-17
...,...,...,...,...,...,...,...
627,2016-10-17 18:30:00,34.692501,135.191315,216.0,number9,POINT (78608.635 -144713.503),2016-10-17
628,2016-10-17 21:00:00,34.698212,135.191330,1080.0,number9,POINT (78604.609 -144079.980),2016-10-17
698,2016-10-17 11:00:00,34.666222,135.213333,684.0,number10,POINT (80651.466 -147611.171),2016-10-17
699,2016-10-17 11:30:00,34.666180,135.213455,612.0,number10,POINT (80662.688 -147615.732),2016-10-17


In [ ]:
# 個人を指定した日付のデータを取得する
df_changed[(df_changed['dt_date'] == date(2016,10,17)) & (df_changed['number'] == 'number1')]

,dt,Latitude,Longitude,Speed,number,geometry,dt_date
0,2016-10-17 11:00:00,34.665825,135.214401,972.0,number1,POINT (80749.735 -147654.354),2016-10-17
1,2016-10-17 12:00:00,34.665089,135.214233,396.0,number1,POINT (80735.052 -147736.132),2016-10-17
2,2016-10-17 12:30:00,34.668037,135.212723,9108.0,number1,POINT (80593.800 -147410.325),2016-10-17
3,2016-10-17 13:30:00,34.665871,135.212769,1152.0,number1,POINT (80600.114 -147650.558),2016-10-17
4,2016-10-17 14:00:00,34.665062,135.211563,0.0,number1,POINT (80490.364 -147741.263),2016-10-17
5,2016-10-17 15:00:00,34.666470,135.212982,6552.0,number1,POINT (80619.056 -147583.942),2016-10-17
6,2016-10-17 15:30:00,34.666351,135.211487,2232.0,number1,POINT (80482.151 -147598.338),2016-10-17
7,2016-10-17 16:00:00,34.685371,135.200699,15192.0,number1,POINT (79475.239 -145497.053),2016-10-17
8,2016-10-17 16:30:00,34.693600,135.196747,576.0,number1,POINT (79105.285 -144587.337),2016-10-17
9,2016-10-17 19:00:00,34.694813,135.193909,0.0,number1,POINT (78844.112 -144455.008),2016-10-17


In [ ]:
## 距離の算出: LineStringにして、length属性を表示
shapely.geometry.LineString(df_changed[(df_changed['dt_date'] == date(2016,10,17)) & (df_changed['number'] == 'number1')]['geometry']).length

5436.153924139514

In [ ]:
# 日付ごとの各個人の移動距離を見る

for date in date_list:
    d_sel_df = df_changed[df_changed['dt_date'] == date]
    print(f'------{date}------')
    for num in d_sel_df['number'].unique():
        num_sel_df = d_sel_df[d_sel_df['number'] == num]
        num_len = shapely.geometry.LineString(num_sel_df['geometry'].values).length
        print(f'名前 {num}: 距離 {num_len}')

------2016-10-17------
名前 number1: 距離 5436.153924139514
名前 number2: 距離 672.4159903243644
名前 number7: 距離 5752.003153671799
名前 number8: 距離 1729.839769030433
名前 number9: 距離 6412.513832718955
名前 number10: 距離 65.30364119330494
------2016-10-18------
名前 number1: 距離 10205.765935618743
名前 number2: 距離 573.2651851029054
名前 number3: 距離 1181.8948156365416
名前 number4: 距離 5025.253469382685
名前 number5: 距離 3533.1521840303153
名前 number6: 距離 745.0471249254642
名前 number7: 距離 9856.986632718257
名前 number8: 距離 1174.6795645013356
名前 number9: 距離 8956.073620775567
名前 number10: 距離 1472.0561280917368
------2016-10-19------
名前 number1: 距離 8544.483731695893
名前 number2: 距離 1122.0347017468928
名前 number3: 距離 1961.6264272645758
名前 number4: 距離 61308.03408909692
名前 number5: 距離 12757.94348927911
名前 number6: 距離 1383.7066944672256
名前 number7: 距離 9003.000099464545
名前 number8: 距離 2.801118825573482
名前 number9: 距離 8249.989251174373
名前 number10: 距離 576.6819511312184
------2016-10-20------
名前 number1: 距離 15775.893859318832
名前 nu

## まとめ

- Pythonを使って、位置データを取得、整形、可視化する作業を行いました
- 位置データがどのように使えるかを体感していただけたかと思います
- 分析作業はここから。会社が持っている課題と、データからのインサイトなどを照らし合わせながら、分析していく。

## イベントアンケートにご協力ください

- 今後のイベントの参考とさせていただくアンケートにご協力ください
- https://forms.gle/DzRvntdQFzN755qdA

## 2回目のイベントも開催決定！

- 次回は国勢調査のデータを位置データと一緒に扱います
- https://techplay.jp/event/832814